In [1]:
# import relevant moduls
import pandas as pd
pd.set_option('display.max_columns', None)
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../scripts/')
from analysis import vectorize_hashtags, compare_vectors

In [2]:
# read and clean files
cluster_df = pd.read_json('../../data/BTW17_Suggestions/suggestions/cluster.json')
topics_df = pd.read_json('../../data/BTW17_Twitter/lda/hashtag_topics.json')

In [3]:
# filter out all noisepoints
cluster_df = cluster_df[cluster_df['cluster']!=-1].reset_index(drop=True)
cluster_df.head(3)

,t-SNE(x),t-SNE(y),suggestion,cluster,vector
0,58.082256,-41.479065,[sebastian],0,"[-0.5116000175000001, -0.3716000021, -0.538900..."
1,58.891510,-41.588104,[bachs],0,"[0.3255000114, 0.1488000005, -0.301699996, -0...."
2,58.082260,-41.479073,[sebastian],0,"[-0.20757141710000002, -0.0776142851, -0.07805..."


In [4]:
topics_df.head(3)

,hashtag,topic,topic_words,scores
0,diesel,71,"[schlussrunde, fast, deshalb, deutsch, rente, ...","[0.428, 0.040, 0.039, 0.033, 0.022, 0.020, 0.0..."
1,denkenwirneu,27,"[völlig, frei, polizei, islam, zeit, verändern...","[0.046, 0.042, 0.041, 0.041, 0.038, 0.030, 0.0..."
2,bundestag,62,"[stoppen, sozial, bekämpfen, mindestlohn, schü...","[0.075, 0.068, 0.064, 0.053, 0.052, 0.044, 0.0..."


In [5]:
hashtag_vectors = vectorize_hashtags(topics_df)

In [6]:
similarity_df = compare_vectors(hashtag_vectors, cluster_df)
similarity_df.head(3)

  0%|          | 0/13171 [00:00<?, ?it/s]

,suggestion,cluster,similarity_scores
0,[sebastian],0,"[0.487, 0.4, 0.645, 0.418, 0.4, 0.435, 0.489, ..."
1,[bachs],0,"[0.321, 0.388, 0.456, 0.476, 0.388, 0.5, 0.508..."
2,[sebastian],0,"[0.563, 0.778, 0.643, 0.731, 0.778, 0.735, 0.6..."


In [7]:
# save df to json
similarity_df.to_json('../../data/BTW17_Suggestions/suggestions/vector_similarity.json')